In [1]:
# 이론의 코드를 실행 가능한 상태로 구현
import os
import warnings
warnings.filterwarnings('ignore')

import numpy as np
from typing import List, Tuple
from dotenv import load_dotenv

# langchain
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter

# BM25
from rank_bm25 import BM25Okapi

# 환경설정
load_dotenv()

True

In [8]:
def check_evnironment():

    '''환경변수 확인'''
    if not os.getenv('OPENAI_API_KEY'):
        raise ValueError('check openai key..')
    print('키 확인 완료')

# 임베딩 기본 개념
def embedding_basic():
    '''텍스트를 수치 벡터로 변환하는 임베딩'''
    # openal 임베딩 모델
    embeddings = OpenAIEmbeddings(model = 'text-embedding-3-small')

    # 단일 텍스트 임베딩
    text = '한국어 임베딩 테스트입니다.'
    vector = embeddings.embed_query(text)

    print(f'입력 테스트 : {text}')
    print(f'벡터차원 : {len(vector)}')

    # 여러 텍스트 배치 임베딩
    texts = [
        'LangGraph는 에이전트 프레임워크입니다.',
        'RAG는 검색 증강 생성입니다.',
        'Python은 프로그래밍 언어입니다.',
    ]
    vectors = embeddings.embed_documents(texts)
    print(f'입력 테스트 : {texts}')
    print(f'벡터차원 : {len(vectors)}')
    print(f'첫 번째 벡터차원 : {vectors[0]}')
    return embeddings


In [ ]:
def cosine_similarity():
    def cosine_similarity(vec1: List[float], vec2: List[float]) -> float:
            """두 벡터의 코사인 유사도 계산"""
            vec1 = np.array(vec1)
            vec2 = np.array(vec2)
            
            dot_product = np.dot(vec1, vec2)
            norm1 = np.linalg.norm(vec1)
            norm2 = np.linalg.norm(vec2)
            
            return dot_product / (norm1 * norm2)
        
        # 임베딩 모델
    embeddings = OpenAIEmbeddings(model='text-embedding-3-small')

    # 테스트 문장들
    sentences = [
        "나는 행복합니다.",           # 기준 문장
        "나는 기쁩니다.",             # 유사한 의미
        "오늘 날씨가 좋습니다.",       # 다른 주제
        "I am happy.",              # 영어 번역
    ]

    # 임베딩 생성
    vectors = [embeddings.embed_query(s) for s in sentences]

    print("\n[코사인 유사도 비교]")
    print(f"   기준 문장: '{sentences[0]}'")
    print()

    base_vector = vectors[0]
    for i, (sentence, vector) in enumerate(zip(sentences[1:], vectors[1:]), 1):
        similarity = cosine_similarity(base_vector, vector)
        print(f"   vs '{sentence}': {similarity:.4f}")

    print("   - 유사도 1.0: 완전히 동일")
    print("   - 유사도 0.8+: 매우 유사")
    print("   - 유사도 0.5+: 어느 정도 관련")
    print("   - 유사도 0.3-: 거의 무관")

    print("\n 코사인 유사도 계산 완료!")
    return cosine_similarity

In [4]:
def bm25_sparse_search():
    '''키워드 기반의 Sparse 검색을 구현'''
    # 문서 데이터
    documents = [
        "LangGraph는 LangChain 위에 구축된 에이전트 프레임워크입니다.",
        "RAG는 Retrieval-Augmented Generation의 약자입니다.",
        "Python은 데이터 과학에서 가장 많이 사용되는 언어입니다.",
        "벡터 데이터베이스는 임베딩을 저장하고 검색합니다.",
        "ChromaDB는 오픈소스 벡터 데이터베이스입니다.",
    ]
    # 간단한 한국어 토큰화(공백 + 조사 분리)
    def simple_korean_tokenize(text:str) ->List[str]:
        '''공백으로 분리 후 각 단어를 2-gram으로 분리'''
        tokens = []
        for word in text.split():
            tokens.append(word)
            # 2글자 이상이면 n-gram도 추가
            if len(word) >=2:
                for i in range(len(word) -1):
                    tokens.append(word[i:i+2])
        return tokens
    # 문서 토큰화
    tokenized_docs = [simple_korean_tokenize(doc) for doc in documents]
    print(f'토큰화....')
    print(f'원본[0] : {documents[0][:30]}...')
    print(f'토큰[0] : {tokenized_docs[0][:10]}...')

    # bm25 인덱스 생성
    bm25 =  BM25Okapi(tokenized_docs)

    # 검색 테스트
    quries = [
        'LangChain 프레임워크',
        '벡터 데이터베이스',
        'Python 프로그래밍'
    ]
    print('\n[BM25 검색 결과]')
    for query in quries:
        tokenized_query = simple_korean_tokenize(query)
        scores = bm25.get_scores(tokenized_query)
        # 상위 2개 결과
        top_indices = np.argsort(scores)[::-1][:2]
        print(f'\n  질문 : {query}')
        for idx in top_indices:
            print(f'    {scores[idx]:.2f}  {documents[idx][:40]}...')
    return bm25,documents,simple_korean_tokenize



In [ ]:
# 하이브리드 검색 구현
def hyprid_search():
    '''Dense(의미 기반)와 Sparse(키워드 기반) 검색을 결합'''
    # 문서데이터 (실제는 전용 Loader를 사용(예/ Textloader PDFLoder 등) 또는 사용자가 직접 수집한 데이터를 Document 객체로 만들어서 리스트 형태
    documents = [
        Document(page_content="LangGraph는 LangChain 위에 구축된 상태 기반 에이전트 프레임워크입니다.", metadata={"id": 1}),
        Document(page_content="RAG(Retrieval-Augmented Generation)는 검색과 생성을 결합한 기술입니다.", metadata={"id": 2}),
        Document(page_content="Python은 데이터 과학과 AI 개발에 널리 사용됩니다.", metadata={"id": 3}),
        Document(page_content="벡터 임베딩은 텍스트를 수치 벡터로 변환합니다.", metadata={"id": 4}),
        Document(page_content="ChromaDB는 로컬에서 사용할 수 있는 벡터 데이터베이스입니다.", metadata={"id": 5}),
    ]

    # Dense 검색 설정
    embeddings = OpenAIEmbeddings(model = 'text_embedding-3-small')
    Chroma.from_documents(
        documents = documents,
        embedding = embeddings,
        collection_name = 'hybrid_example'
    )
    dense_retriever = vectorstore.as_retriever(search_kwargs={'k':3})
    
     # 간단한 한국어 토큰화(공백 + 조사 분리)
    def simple_korean_tokenize(text:str) ->List[str]:
        '''공백으로 분리 후 각 단어를 2-gram으로 분리'''
        tokens = []
        for word in text.split():
            tokens.append(word)
            # 2글자 이상이면 n-gram도 추가
            if len(word) >=2:
                for i in range(len(word) -1):
                    tokens.append(word[i:i+2])
        return tokens 
    doc_text = [doc.page_content for doc in documents]  
    tokenized_docs = [simple_korean_tokenize(text) for text in doc_texts]
    bm25 = BM25Okapi(tokenized_docs)

    def sparse_search(query:str, k:int=3) -> List[Document]:
        '''bm25 기반 검색'''
        tokenized_query = simple_korean_tokenize(query)
        scores = bm25.get_scores(tokenized_query)
        top_indices = np.argsort(scores)[::-1][:k]
        return [documents[i] for i in top_indices]

    # 하이브리드 검색
    def hybrid_search(query:str, dense_weight: float = 0.7, k:int=3) -> List[Document]:
        '''Dense + Sparse 하이브리드 검색
        Args:
            query : 검색
            dense_weight : Dense 검색 가중치
            k : 반환할 문서 수
        '''
        sparse_weight = 1 - dense_weight
        # Dense 검색
        dense_result = dense_retriever.invoke(query)
        # Sparse 검색
        sparse_results = sparse_search(query, k=k)
        # RRF(Reciprocal Rank Fusion) 점수 계산
        doc_scores = {}
        for rank,doc in enumerate(dense_result):
            doc_id = doc.metadata.get('id')
            if doc_id not in doc_scores:
                doc_scores[doc_id] = {'doc':doc, 'score':0}
            doc_scores[doc_id]['score'] += dense_weight + (1/(rank+1))
        for rank,doc in enumerate(sparse_results):
            doc_id = doc.metadata.get('id')
            if doc_id not in doc_scores:
                doc_scores[doc_id] = {'doc':doc, 'score':0}
            doc_scores[doc_id]['score'] += sparse_weight + (1/(rank+1))
        # 점수순 정렬
        sorted_results = sorted(  doc_scores.values(), key = lambda x: x['score'], reverse=True  )
        return [ item["doc"] for item in sorted_results[:k]]
    # --- 테스트 ---
    test_queries = [
        "에이전트 프레임워크",     # Dense에 유리 (의미)
        "ChromaDB",              # Sparse에 유리 (정확한 키워드)
        "RAG 검색 생성"          # 하이브리드에 유리
    ]
    print("\n[검색 방식별 비교]")
    for query in test_queries:
        print(f"\n 질문: '{query}'")
        
        # Dense만
        dense_results = dense_retriever.invoke(query)
        print(f"   [Dense] {dense_results[0].page_content[:40]}...")
        
        # Sparse만
        sparse_results = sparse_search(query, k=1)
        print(f"   [Sparse] {sparse_results[0].page_content[:40]}...")
        
        # 하이브리드
        hybrid_results = hybrid_search(query, dense_weight=0.7, k=1)
        print(f"   [Hybrid] {hybrid_results[0].page_content[:40]}...")
    
    # 정리
    vectorstore.delete_collection()
    
    print("\n 하이브리드 검색 완료!")
    return hybrid_search

def embedding_evaluation():
    """임베딩 품질 평가"""    
    embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
    
    # 테스트 데이터: (질문, 관련 문서, 비관련 문서)
    test_pairs = [
        (
            "Python 코드 작성법",
            "Python은 간결한 문법으로 코드를 작성할 수 있는 프로그래밍 언어입니다.",
            "오늘 날씨가 매우 좋습니다."
        ),
        (
            "한국어 임베딩 모델",
            "BGE-M3는 한국어를 포함한 다국어 임베딩을 지원합니다.",
            "피자는 이탈리아 음식입니다."
        ),
        (
            "RAG 시스템 구축",
            "RAG는 검색과 생성을 결합하여 더 정확한 답변을 제공합니다.",
            "축구는 세계에서 가장 인기 있는 스포츠입니다."
        ),
    ]
    
    def cosine_sim(vec1, vec2):
        """코사인 유사도"""
        vec1 = np.array(vec1)
        vec2 = np.array(vec2)
        return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))
    
    def evaluate_embedding_model(test_data: List[Tuple[str, str, str]]) -> dict:
        """
        임베딩 모델 품질 평가
        
        평가 기준:
        - 관련 문서가 비관련 문서보다 유사도가 높으면 정답
        """
        correct = 0
        results = []
        
        for query, positive, negative in test_data:
            q_vec = embeddings.embed_query(query)
            p_vec = embeddings.embed_query(positive)
            n_vec = embeddings.embed_query(negative)
            
            pos_sim = cosine_sim(q_vec, p_vec)
            neg_sim = cosine_sim(q_vec, n_vec)
            
            is_correct = pos_sim > neg_sim
            if is_correct:
                correct += 1
            
            results.append({
                "query": query[:20],
                "pos_sim": pos_sim,
                "neg_sim": neg_sim,
                "correct": is_correct
            })
        
        accuracy = correct / len(test_data)
        return {"accuracy": accuracy, "details": results}
    
    # 평가 실행
    eval_result = evaluate_embedding_model(test_pairs)
    
    print("\n[임베딩 품질 평가 결과]")
    print(f"   정확도: {eval_result['accuracy']:.1%}")
    print()
    
    for detail in eval_result["details"]:
        status = "ok" if detail["correct"] else "bed"
        print(f"   {status} '{detail['query']}...'")
        print(f"      관련 유사도: {detail['pos_sim']:.4f}")
        print(f"      비관련 유사도: {detail['neg_sim']:.4f}")
    
    print("\n   해석:")
    print("   - 정확도 90%+: 우수한 임베딩 모델")
    print("   - 정확도 70%+: 사용 가능")
    print("   - 정확도 70%-: 다른 모델 검토 필요")
    
    print("\n 임베딩 품질 평가 완료!")
    return evaluate_embedding_model

if __name__ == '__main__':
    check_evnironment()
    # embedding_basic()
    # cosine_simularity()
    # bm25_sparse_search()
    # hybrid_search()
    embedding_evaluation()

키 확인 완료
입력 테스트 : 한국어 임베딩 테스트입니다.
벡터차원 : 1536
입력 테스트 : ['LangGraph는 에이전트 프레임워크입니다.', 'RAG는 검색 증강 생성입니다.', 'Python은 프로그래밍 언어입니다.']
벡터차원 : 3
첫 번째 벡터차원 : [-0.03749444708228111, 0.002410395536571741, 0.015778109431266785, -0.015949610620737076, 0.014481131918728352, -0.014877728186547756, -0.053894247859716415, 0.01700005494058132, -0.015220730565488338, -0.015906736254692078, 0.0003098410088568926, -0.013998785056173801, 0.015338637866079807, -0.010241837240755558, 0.006050776224583387, -0.03845914080739021, -0.026668434962630272, -0.004879744723439217, -0.004260732792317867, 0.013227029703557491, -0.022745344787836075, 0.019465385004878044, 0.024824798107147217, 0.03174915909767151, 0.015928173437714577, 0.0074281455017626286, 0.013816564343869686, -0.014191723428666592, -0.016710646450519562, -0.0019843224436044693, 0.01642123982310295, -0.030827339738607407, 0.03526493161916733, -0.0135271567851305, 0.04476181045174599, 0.022209404036402702, 0.009614785201847553, -0.001372009515762329, 